<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>

# AI Agents & Automation — Chapter 15
## Case Study — Financial Signals & Compliance

&copy; Dr. Yves J. Hilpisch<br>
AI-Powered by GPT-5.

### Overview

This notebook accompanies Chapter 15 — Financial Signals. It is self-contained and demonstrates the core ideas with small, readable code cells. Run cells from top to bottom; each code cell is preceded by a short explanation of what it does.


Deterministic cues→signals with an allowlist, size cap, and compact audits. Swap Pandas later; keep this notebook dependency-free.

In [ ]:
import json  # import JSON utilities
import time  # import time utilities

POSITIVE = {'beats', 'surge', 'record'}  # positive cues
NEGATIVE = {'misses', 'downgrade', 'cut'}  # negative cues
ALLOWLIST = {'ACME', 'FOO', 'BAR'}  # tradable tickers

def cue(text: str) -> str:  # classify sentiment cue
    lower = text.lower()  # normalize text
    if any(word in lower for word in POSITIVE):
        return 'positive'  # upbeat signal
    if any(word in lower for word in NEGATIVE):
        return 'negative'  # downbeat signal
    return 'neutral'  # no signal

def decide(
    ticker: str,
    cue_label: str,
    cap: float = 0.5,
) -> dict[str, str]:
    # apply policy guard + sizing
    if ticker not in ALLOWLIST:
        return {
            'status': 'error',
            'signal': 'n/a',
            'size': '0',
            'reason': 'unknown',
        }  # reject
    if cue_label == 'positive':
        direction = 'up'
    elif cue_label == 'negative':
        direction = 'down'
    else:
        direction = 'flat'  # map cue
    base = 0.5 if direction in {'up', 'down'} else 0.0  # base size
    size_val = min(base, cap)  # respect cap
    reason = 'ok' if size_val == base else 'clamped'  # capture reason
    size_str = '{:.2f}'.format(size_val)  # formatted size
    return {
        'status': 'ok',
        'signal': direction,
        'size': size_str,
        'reason': reason,
    }  # allocation

notes = [
    ('ACME', 'ACME beats estimates'),
    ('FOO', 'FOO misses guidance'),
    ('XYZ', 'unknown ticker'),
]  # sample inputs
for ticker, text in notes:  # stream through sample notes
    decision = decide(ticker, cue(text), cap=0.4)  # evaluate note
    row = [ticker, decision['signal'], decision['size'], decision['reason']]
    print('CSV', ','.join(row))  # CSV-style log
    audit = {
        'ts': int(time.time()),
        'ticker': ticker,
        'signal': decision['signal'],
        'size': decision['size'],
        'reason': decision['reason'],
    }  # audit record
    print('AUD', json.dumps(audit))  # audit log


<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>